# Projet 5: Segmentez les clients d'un site e-commerce (analyse exploratoire)

Objectifs: 
- comprendre les différents **types d'utilisateurs** grâce à leur **comportement** et **données personnelles** => utilisation de méthodes **non supervisées** pour regrouper les clients de profils similaires
- créer une **segmentation des clients** que la société Olist pourra utiliser pour ses **campagnes de communication**. La segmentation proposée doit être exploitable et facile d’utilisation par l'équipe Marketing. Elle doit au minimum pouvoir différencier les **bons et moins bons clients en termes de commandes et de satisfaction**.
- fournir à l’équipe marketing une **description actionable de la segmentation et de sa logique sous-jacente** pour une utilisation optimale, ainsi qu’une proposition de **contrat de maintenance** basée sur une analyse de la **stabilité des segments au cours du temps**. Une recommandation de fréquence à laquelle la segmentation doit être mise à jour pour rester pertinente doit être faite afin de pouvoir rédiger le devis de contrat de maintenance.

## Table des matières: <a class="anchor" id="0"></a>

1. [Import des librairies et configurations générales](#library)
2. [Chargement et visualisation des données](#data)
3. [Consolidation des données (par order_id)](#conso)
4. [Consolidation des données (par customer_unique_id)](#tab_cli)

## Import des librairies et configurations générales <a class="anchor" id="library"></a>

In [ ]:
# Pour vérifier si la convention PEP8 est respectée
%load_ext pycodestyle_magic
%pycodestyle_on

In [ ]:
# import warnings
# warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns

import unidecode  # caractères spéciaux

import datetime as dt

from scipy.stats import chi2_contingency

# Configuration générale des graphiques
parameters = {'axes.labelsize': 13,
              'axes.titlesize': 16,
              'axes.titleweight': 'bold'}
plt.rcParams.update(parameters)

# Modification de l'affichage des lignes et colonnes pour plus de lisibilité
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', None)
# pd.options.display.float_format = '{:20,.2f}'.format

## Chargement et visualisation des données <a class="anchor" id="data"></a>

Olist nous fournit une **base de données anonymisée** comportant des informations sur:

- l’historique de commandes, 
- les produits achetés, 
- les commentaires de satisfaction, 
- et la localisation des clients depuis janvier 2017

Seuls les 3% des clients qui ont réalisé plusieurs commandes nous intéressent.

A noter que le code fourni doit respecter la convention PEP8 pour être exploité par Olist. Quelques exemples de conventions à respecter:
- encodage UTF-8
- indentation de 4 caractères
- 79 caractères par ligne
- imports à déclarer au début du script

Nous allons dans cette partie charger les données de chaque dataset afin de comprendre quelles sont celles qui pourront nous être utiles pour identifier les différents types d'utilisateurs.

Nous vérifierons les également les lignes qui apparaissent plusieurs fois sur les clefs de jointures.

In [ ]:
# Import des datasets
customers = pd.read_csv('data/olist_customers_dataset.csv',
                        sep=',',
                        encoding='utf-8')
customers.name = 'customers'

geo = pd.read_csv('data/olist_geolocation_dataset.csv',
                  sep=',',
                  encoding='utf-8')
geo.name = 'geo'

items = pd.read_csv('data/olist_order_items_dataset.csv',
                    sep=',',
                    encoding='utf-8')
items.name = 'items'

payments = pd.read_csv('data/olist_order_payments_dataset.csv',
                       sep=',',
                       encoding='utf-8')
payments.name = 'payments'

reviews = pd.read_csv('data/olist_order_reviews_dataset.csv',
                      sep=',',
                      encoding='utf-8')
reviews.name = 'reviews'

orders = pd.read_csv('data/olist_orders_dataset.csv',
                     sep=',',
                     encoding='utf-8')
orders.name = 'orders'

products = pd.read_csv('data/olist_products_dataset.csv',
                       sep=',',
                       encoding='utf-8')
products.name = 'products'

sellers = pd.read_csv('data/olist_sellers_dataset.csv',
                      sep=',',
                      encoding='utf-8')
sellers.name = 'sellers'

category = pd.read_csv('data/product_category_name_translation.csv',
                       sep=',',
                       encoding='utf-8')
category.name = 'category'

**Architecture des données**
<img src="data\Data_Scheme.PNG">

### olist_customers_dataset.csv

Ce jeu de données contient des informations sur le **client** et son **emplacement**. Il permet **d'identifier les clients uniques dans l'ensemble de données des commandes** et pour trouver le **lieu de livraison des commandes**.

Le même client aura des identifiants différents pour des commandes différentes. Le but d'avoir un customer_unique_id dans l'ensemble de données est de permettre **d'identifier les clients qui ont fait des réachats au magasin**.

In [ ]:
# Visualisation des 5 premières lignes du dataset
customers.head()

In [ ]:
def shape_total_nan(dataframe):
    '''Fonction qui retourne le nombre de lignes,
    de variables, le nombre total de valeurs manquantes et
    le pourcentage associé'''
    missing = dataframe.isna().sum().sum()
    missing_percent = round(missing
                            / (dataframe.shape[0] * dataframe.shape[1])
                            * 100,
                            2)

    print(f"Nombre de lignes: {dataframe.shape[0]}")
    print(f"Nombre de colonnes: {dataframe.shape[1]}")
    print(f"Nombre total de NaN du dataset: {missing}")
    print(f"% total de NaN du dataset: {missing_percent}%")

In [ ]:
shape_total_nan(customers)

In [ ]:
def describe_variables(data):
    ''' Fonction qui prend un dataframe en entrée, et retourne un
    récapitulatif qui contient le nom des variables, leur type, un
    exemple de modalité, le nombre total de lignes, le nombre et
    pourcentage de valeurs distinctes, le nombre et pourcentage de
    valeurs non manquantes et de valeurs manquantes (NaN) et les
    principales statistiques pour les variables numériques (moyenne,
    médiane, distribution, variance, écart type, minimum, quartiles et
    maximum)'''

    # Choix du nom des variables à afficher
    df = pd.DataFrame(columns=[
        'Variable name', 'Variable type', 'Example', 'Raws', 'Distinct',
        '% distinct', 'Not NaN', '% Not NaN', 'NaN', '% NaN', 'Mean',
        'Median', 'Skew', 'Kurtosis', 'Variance', 'Std', 'Min', '25%',
        '75%', 'Max'
    ])

    # Pour chaque colonne du dataframe
    for col in data.columns:

        # Définition des variables
        # type de la variable (object, float, int...)
        var_type = data[col].dtypes
        # premier élément notNA
        example = data[data[col].notna()][col].iloc[0]
        # nombre total de lignes
        nb_raw = len(data[col])
        # nombre de valeurs non manquantes
        count = len(data[col]) - data[col].isna().sum()
        # % de valeurs non manquantes
        percent_count = round(data[col].notnull().mean(), 4)*100
        # nombre de modalités que peut prendre la variable
        distinct = data[col].nunique()
        # % de valeurs distinctes
        percent_distinct = round(data[col].nunique()/len(data[col]), 4)
        percent_distinct = percent_distinct * 100
        # nombre de valeurs manquantes
        missing = data[col].isna().sum()
        # % de valeurs manquantes
        percent_missing = round(data[col].isna().mean(), 4)*100

        # Pour les var de type 'int' ou 'float': on remplit toutes les col
        if var_type == 'int32' or var_type == 'int64' or var_type == 'float':
            df = pd.concat([df, pd.DataFrame([[col, var_type, example, nb_raw,
                                               distinct, percent_distinct,
                                               count,
                                               percent_count,
                                               missing,
                                               percent_missing,
                                               round(data[col].mean(), 2),
                                               round(data[col].median(), 2),
                                               round(data[col].skew(), 2),
                                               round(data[col].kurtosis(), 2),
                                               round(data[col].var(), 2),
                                               round(data[col].std(), 2),
                                               round(data[col].min(), 2),
                                               round(data[col].quantile(0.25),
                                                     2),
                                               round(data[col].quantile(0.75),
                                                     2),
                                               data[col].max()]],
                                             columns=['Variable name',
                                                      'Variable type',
                                                      'Example',
                                                      'Raws',
                                                      'Distinct',
                                                      '% distinct',
                                                      'Not NaN',
                                                      '% Not NaN',
                                                      'NaN',
                                                      '% NaN',
                                                      'Mean',
                                                      'Median',
                                                      'Skew',
                                                      'Kurtosis',
                                                      'Variance',
                                                      'Std',
                                                      'Min',
                                                      '25%',
                                                      '75%',
                                                      'Max'])])

            # Pour les variables d'un autre type: on ne remplit que
            # les variables de compte

        else:
            df = pd.concat([df, pd.DataFrame([[col, var_type, example,
                                               nb_raw, distinct,
                                               percent_distinct,
                                               count,
                                               percent_count, missing,
                                               percent_missing,
                                               '', '', '', '', '', '',
                                               '', '', '', '']],
                                             columns=['Variable name',
                                                      'Variable type',
                                                      'Example',
                                                      'Raws',
                                                      'Distinct',
                                                      '% distinct',
                                                      'Not NaN',
                                                      '% Not NaN',
                                                      'NaN',
                                                      '% NaN',
                                                      'Mean',
                                                      'Median',
                                                      'Skew',
                                                      'Kurtosis',
                                                      'Variance',
                                                      'Std',
                                                      'Min',
                                                      '25%',
                                                      '75%',
                                                      'Max'])])

    return df.reset_index(drop=True)

In [ ]:
describe_variables(customers)

In [ ]:
# Nombre de doublons totaux
print(f"Lignes en doublons: {customers.duplicated().sum()}")

In [ ]:
# Nombre de commandes par customer_unique_id
customers_orders = customers["customer_unique_id"].value_counts()

print("Nb de commandes par client (> 2 commandes)")
customers_orders[customers_orders > 1]

Observations:

- aucune valeur manquante
- la variable customer_id ne comporte que des éléments uniques
- certains customer_unique_id apparaissent sur plusieurs lignes du dataset correspondant à des commandes différentes
- ce dataset est relié au dataset geolocation par la clef zip_code_prefix et au dataset orders par la clef customer_id%

### olist_geolocation_dataset.csv

Ce jeu de données contient des informations sur les **codes postaux** brésiliens et leurs coordonnées **lat/lng**. Il permet de calculer la distance entre le vendeur et le client.

In [ ]:
# Visualisation des 5 premières lignes du dataset
geo.head()

In [ ]:
# Shape du dataset
shape_total_nan(geo)

In [ ]:
# Description des variables
describe_variables(geo)

In [ ]:
# Doublons totaux
print(f"Lignes en doublons: {geo.duplicated().sum()}")

In [ ]:
# Suppression des doublons intégraux
geo = geo.drop_duplicates()
geo.shape

In [ ]:
# Nombre de lignes ayant le même zip_code_prefix
print(f"Nb lignes: {geo.duplicated('geolocation_zip_code_prefix').sum()}")

# Récupération des zip_code_prefix qui apparaissent plusieurs fois
zip_code_dup = geo[geo.duplicated('geolocation_zip_code_prefix')][
    'geolocation_zip_code_prefix'].unique()

# Filtrage du dataset sur ces zip_code_prefix
geo_dup = geo[geo['geolocation_zip_code_prefix'].isin(
    zip_code_dup)].sort_values(
    by='geolocation_zip_code_prefix')
geo_dup

Un même zip_code prefix peut avoir différentes latitudes et longitudes. Afin d'avoir des zip_codes uniques, nous allons calculer la moyenne des latitudes et longitudes par code postal puis récupérer la ville et l'état:

In [ ]:
# Moyenne des lat et lon par code postal
zip_mean = geo.groupby('geolocation_zip_code_prefix')[
    ['geolocation_lat', 'geolocation_lng']].mean()

# On conserve une ville par code postal
zip_city = geo[['geolocation_zip_code_prefix',
                'geolocation_city',
                'geolocation_state']].groupby(
    ['geolocation_zip_code_prefix',
     'geolocation_city',
     'geolocation_state']).count().reset_index()

zip_city = zip_city.drop_duplicates('geolocation_zip_code_prefix',
                                    keep='first')

# Merge
geo = pd.merge(left=zip_mean, right=zip_city,
               how='left', on='geolocation_zip_code_prefix')

Le dataset contenant 27 états distincts, nous allons réduire le nombre de modalités en les regroupant en zones.

In [ ]:
# Modalités de geolocation_state
geo['geolocation_state'].unique()

In [ ]:
# Création du dictionnaire des zones
ZONES = {
    'PA': 'Nord',
    'AP': 'Nord',
    'RR': 'Nord',
    'SE': 'Nord_Est',
    'PE': 'Nord_Est',
    'RN': 'Nord_Est',
    'AL': 'Nord_Est',
    'PB': 'Nord_Est',
    'CE': 'Nord_Est',
    'PI': 'Nord_Est',
    'MA': 'Nord_Est',
    'AM': 'Nord_Ouest',
    'AC': 'Nord_Ouest',
    'DF': 'Centre',
    'GO': 'Centre',
    'TO': 'Centre',
    'MT': 'Centre',
    'BA': 'Centre_Est',
    'RO': 'Centre_Ouest',
    'PR': 'Sud',
    'SC': 'Sud',
    'RS': 'Sud',
    'SP': 'Sud_Est',
    'RJ': 'Sud_Est',
    'ES': 'Sud_Est',
    'MG': 'Sud_Est',
    'MS': 'Sud_Ouest',
}

In [ ]:
def detect_words(values, dictionary):
    result = []
    for lib in values:
        categ = "AUTRE"
        for word, val in dictionary.items():
            if word in lib:
                categ = val
        result.append(categ)
    return result


geo["zone"] = detect_words(geo["geolocation_state"], ZONES)

In [ ]:
# Création de tuples lat / lon
geo['lat_lon'] = list(zip(geo['geolocation_lat'], geo['geolocation_lng']))

In [ ]:
# geo['point'] = geo.apply(lambda row: Point(latitude=row['geolocation_lat'],
#                                           longitude=row['geolocation_lng']),
#                         axis=1)

In [ ]:
geo.head()

Observations:

- aucune valeur manquante
- un grand nombre de lignes en doublons qui ont été supprimées
- d'après l'architecture des données, le zip_code_prefix est la clef qui fait le lien avec les datasets customers et sellers
- ce dataset est dorénavant composé d'une observation par code postal
- un colonne 'zone' avec moins de modalités et regroupant plusieurs states a été crééé

### olist_order_items_dataset.csv

Ce dataset inclut les informations sur éléments achetés de chaque commande.

In [ ]:
# Visualisation des premières lignes du dataset
items.sort_values(by='product_id').head(10)

In [ ]:
# Shape du dataset
shape_total_nan(items)

In [ ]:
# Description des variables
describe_variables(items)

In [ ]:
# Doublons totaux
print(f"Lignes en doublons: {items.duplicated().sum()}")

In [ ]:
# Nombre de lignes ayant le même order_id
print(f"Nb lignes: {items.duplicated(['order_id']).sum()}")

# Récupération des order_id qui apparaissent plusieurs fois
order_id_dup = items[items.duplicated('order_id')][
    'order_id'].unique()

# Filtrage du dataset sur ces order_id
items_dup = items[items['order_id'].isin(
    order_id_dup)].sort_values(
    by='order_id')
items_dup

Un même order_id peut apparaitre plusieurs fois, c'est à dire à chaque fois qu'un article est rajouté dans la commande. Il ne s'agit pas de doublons.

Observations:

- aucune valeur manquante
- d'après l'architecture des données, le seller_id est la clef qui fait le lien avec le dataset sellers, order_id avec le dataset orders et product_id avec le dataset products
- il existe plusieurs lignes pour un même order_id, il s'agit des différents produits rajoutés dans la commande

Afin de pouvoir ensuite merger cette table avec celle des commandes tout en ayant une observation par commande, nous allons regrouper les lignes par order_id et créer les indicateurs qui nous intéressent:
- nombre d'articles achetés
- nombre d'articles différents achetés
- produit le plus acheté
- nombre de vendeurs
- vendeur préféré
- prix total des articles
- prix du transport
- prix de l'article le moins cher
- prix de l'article le plus cher

In [ ]:
items_merge = items.groupby(['order_id'],
                            as_index=False,
                            dropna=False).agg(
    nb_art=('order_item_id', 'max'),
    nb_art_diff=('product_id', 'nunique'),
    art_pref=('product_id', lambda x: x.mode()[0]),
    nb_vendeurs=('seller_id', 'nunique'),
    vendeur_pref=('seller_id', lambda x: x.mode()[0]),
    prix_tot_art=('price', 'sum'),
    prix_transport=('freight_value', 'sum'),
    prix_art_min=('price', 'min'),
    prix_art_max=('price', 'max'))

In [ ]:
# Vérification que la table contient bien une ligne par commande
items_merge.shape[0] == items['order_id'].nunique()

In [ ]:
items_merge.head()

### olist_order_payments_dataset.csv

Ce dataset contient les informations sur les options de paiement.

In [ ]:
# Visualisation des 5 premières lignes du dataset
payments.head()

In [ ]:
# Shape du dataset
shape_total_nan(payments)

In [ ]:
# Description des variables
describe_variables(payments)

In [ ]:
# Doublons totaux
print(f"Lignes en doublons: {payments.duplicated().sum()}")

In [ ]:
# Nombre de lignes ayant le même order_id
print(f"Nb lignes: {payments.duplicated(['order_id']).sum()}")

# Récupération des order_id qui apparaissent plusieurs fois
order_id_dup = payments[payments.duplicated('order_id')][
    'order_id'].unique()

# Filtrage du dataset sur ces order_id
payments_dup = payments[payments['order_id'].isin(
    order_id_dup)].sort_values(
    by='order_id')
payments_dup

Une même commande peut apparaitre sur plusieurs lignes. C'est le cas lorsque le client a payé avec 2 méthodes différentes. La variable payment_installments correspond au nombre de versements effectués par type de paiment. Il ne s'agit donc pas de doublons.

Observations:

- aucune valeur manquante
- d'après l'architecture des données, l'order_id est la clef qui fait le lien avec le dataset orders
- order_id apparait sur plusieurs lignes lorsque le client a utilisé différents moyens de paiment

Afin de pouvoir ensuite merger cette table avec celle des commandes tout en ayant une observation par commande, nous allons regrouper les lignes par order_id et créer les indicateurs qui nous intéressent:
- nombre maximum de facilités de paiement par commande
- moyen de paiement préféré

In [ ]:
payments_pivot = payments.pivot(index=['order_id',
                                       'payment_sequential',
                                       'payment_installments'],
                                columns='payment_type',
                                values='payment_value').reset_index()

In [ ]:
payments_merge = payments_pivot.groupby(['order_id'],
                                        as_index=False,
                                        dropna=False).agg(
    facil_paiement_max=('payment_installments', 'max'),
    boleto=('boleto', 'sum'),
    credit_card=('credit_card', 'sum'),
    debit_card=('debit_card', 'sum'),
    not_defined=('not_defined', 'sum'),
    voucher=('voucher', 'sum'))

payments_merge.head()

In [ ]:
# Vérification que la table contient bien une ligne par commande
payments_merge.shape[0] == payments['order_id'].nunique()

In [ ]:
payments_merge['moy_paiement_pref'] = payments_merge[['boleto',
                                                      'credit_card',
                                                      'debit_card',
                                                      'not_defined',
                                                      'voucher']].idxmax(
    axis='columns')

In [ ]:
payments_merge = payments_merge[['order_id', 'facil_paiement_max',
                                 'moy_paiement_pref']]
payments_merge.head()

### olist_order_reviews_dataset.csv

Ce dataset contient les informations sur les commentaires des clients.

Après qu'un client ait acheté un produit sur Olist Store, le vendeur est informé qu'il doit exécuter la commande. Une fois que le client a reçu le produit, ou que la date de livraison estimée est arrivée, le client reçoit une enquête de satisfaction par e-mail où il peut donner une note sur l'expérience d'achat et écrire quelques commentaires.

In [ ]:
# Visualisation des 5 premières lignes du dataset
reviews.head()

In [ ]:
# Shape du dataset
shape_total_nan(reviews)

In [ ]:
# Description des variables
describe_variables(reviews)

In [ ]:
# Doublons totaux
print(f"Lignes en doublons: {reviews.duplicated().sum()}")

In [ ]:
# Doublons sur la clef order_id
print(f"Nb lignes: {reviews.duplicated('order_id').sum()}")

# Récupération des order_id qui apparaissent plusieurs fois
order_id_dup = reviews[reviews.duplicated('order_id')]['order_id'].unique()

# Filtrage du dataset sur ces order_id
reviews_dup = reviews[reviews['order_id'].isin(order_id_dup)].sort_values(
    by='order_id')
reviews_dup

Une même clef de jointure order_id existe plusieurs fois dans le dataset. Nous allons essayer de comprendre et de trouver une stratégie pour traiter ces doublons.

Stratégie de traitement des order_id_dupliqués:
- Completion des variables review_comment_message et review_comment_title NaN lorsque l'information existe sur une autre ligne
- On garde la date du commentaire la plus récente (review_answer_timestamp) 

In [ ]:
def mapping(dataframe, var_to_map, var_mapping):
    table_mapping = dataframe.groupby(
        [var_mapping, var_to_map])['review_id'].count().reset_index()
    table_mapping = table_mapping.sort_values(var_to_map,
                                              ascending=True)  # sur col txt
    table_mapping = table_mapping.drop_duplicates(subset=var_mapping,
                                                  # first car nan after
                                                  keep='first')

    # Merge du dataframe avec la table de mapping
    df = pd.merge(left=dataframe, right=table_mapping[[var_mapping,
                                                       var_to_map]],
                  how="left", on=var_mapping)

    # Lorsque la var_to_map est NaN, je complète avec la valeur du mapping
    df[f'{var_to_map}_x'] = np.where(df[f'{var_to_map}_x'].isnull(),
                                     df[f'{var_to_map}_y'],
                                     df[f'{var_to_map}_x'])

    # On supprime la variable issue de la jointure et on enlève le suffixe _x
    df.rename(columns={f'{var_to_map}_x': var_to_map}, inplace=True)
    df.drop(f'{var_to_map}_y', axis='columns', inplace=True)

    return df

In [ ]:
# Completion des variables ayant le même order_id
reviews = mapping(reviews, 'review_comment_message', 'order_id')
reviews = mapping(reviews, 'review_comment_title', 'order_id')

In [ ]:
# Suppression des order_id dupliqués en conservant celui
# qui a le commentaire le plus récent
reviews = reviews.sort_values(by='review_answer_timestamp',
                              ascending=False).drop_duplicates('order_id',
                                                               keep='first')

# Doublons sur la clef order_id
print(f"Lignes en doublons: {reviews.duplicated('order_id').sum()}")

Observations:

- valeurs manquantes sur les variables sur les titres et commentaires des questionnaires de satisfaction envoyés aux clients
- d'après l'architecture des données, l'order_id est la clef qui fait le lien avec le dataset orders
- doublons sur l'order_id retraités en completant les valeurs manquantes puis conservant la date de commentaire la plus récente

### olist_orders_dataset.csv

Dataset principal sur les commandes passées par les différents clients.

In [ ]:
# Visualisation des 5 premières lignes du dataset
orders.head()

In [ ]:
# Shape du dataset
shape_total_nan(orders)

In [ ]:
# Description des variables
describe_variables(orders)

In [ ]:
# Doublons totaux
print(f"Lignes en doublons: {orders.duplicated().sum()}")

Observations:

- quelques valeurs manquantes sur les variables order_approved_at, order_delivered_carrier_date et order_delivered_customer_date ce qui semble cohérent. En effet, une commande peut ne pas avoir été validée ou livrée.
- d'après l'architecture des données, l'order_id est la clef qui fait le lien avec les datasets order_payments, order_reviews et order_items => pas de doublons
- customer_id est la clef qui fait le lien avec order_customer => pas de doublons

### olist_products_dataset.csv

Dataset rassemblant tous les produits vendus par Olist.

In [ ]:
# Visualisation des 5 premières lignes du dataset
products.head()

In [ ]:
# Shape du dataset
shape_total_nan(products)

In [ ]:
# Description des variables
describe_variables(products)

In [ ]:
# Doublons totaux
print(f"Lignes en doublons: {products.duplicated().sum()}")

Observations:

- quelques valeurs manquantes
- product_id est la clef qui fait le lien avec le dataset products => pas de doublons

Nous ne conservons pour le merge avec la table consolidée que les variables product_id et product_category_name:

In [ ]:
products_merge = products[['product_id', 'product_category_name']]

### olist_sellers_dataset.csv

Informations sur les vendeurs.

In [ ]:
# Visualisation des 5 premières lignes du dataset
sellers.head()

In [ ]:
# Shape du dataset
shape_total_nan(sellers)

In [ ]:
# Description des variables
describe_variables(sellers)

In [ ]:
# Doublons totaux
dblon = sellers.duplicated(subset=['seller_id',
                                   'seller_zip_code_prefix']).sum()
print(f"Doublons: {dblon}")

Observations:

- aucune valeur manquante
- seller_id est la clef qui fait le lien avec le dataset order_items et zip_code_prefix avec le dataset geolocation

### product_category_name_translation.csv

Traduction des categories des produits en anglais.

In [ ]:
# Visualisation des 5 premières lignes du dataset
category.head()

In [ ]:
# Shape du dataset
shape_total_nan(category)

In [ ]:
# Description des variables
describe_variables(category)

In [ ]:
# Doublons totaux
print(f"Lignes en doublons: {category.duplicated().sum()}")

In [ ]:
# Différentes catégories en anglais
category['product_category_name_english'].unique()

Il existe 71 catégories que nous allons réaffecter dans de plus grandes catégories pour réduire les modalités.

In [ ]:
# Création du dictionnaire des catégories générales
BIG_CAT = {
    'health_beauty': 'hygiene/beauty',
    'computers_accessories': 'electronics',
    'auto': 'auto',
    'bed_bath_table': 'house/decoration/furnitures',
    'furniture_decor': 'house/decoration/furnitures',
    'sports_leisure': 'books/culture/leisure',
    'perfumery': 'hygiene/beauty',
    'housewares': 'house/decoration/furnitures',
    'telephony':  'electronics',
    'watches_gifts': 'electronics',
    'food_drink': 'food',
    'baby': 'baby',
    'stationery': 'house/decoration/furnitures',
    'tablets_printing_image': 'electronics',
    'toys': 'toys',
    'fixed_telephony': 'electronics',
    'garden_tools': 'DoItYourself',
    'fashion_bags_accessories': 'fashion/luggage',
    'small_appliances': 'electronics',
    'consoles_games': 'electronics',
    'audio': 'multimedia',
    'fashion_shoes': 'fashion/luggage',
    'cool_stuff': 'other',
    'luggage_accessories': 'fashion/luggage',
    'air_conditioning': 'electronics',
    'construction_tools_construction': 'DoItYourself',
    'kitchen_dining_laundry_garden_furniture': 'house/decoration/furnitures',
    'costruction_tools_garden': 'DoItYourself',
    'fashion_male_clothing': 'fashion/luggage',
    'pet_shop': 'pets',
    'office_furniture': 'house/decoration/furnitures',
    'market_place': 'other',
    'electronics': 'electronics',
    'home_appliances': 'electronics',
    'party_supplies': 'house/decoration/furnitures',
    'home_confort': 'house/decoration/furnitures',
    'costruction_tools_tools': 'DoItYourself',
    'agro_industry_and_commerce': 'other',
    'furniture_mattress_and_upholstery': 'house/decoration/furnitures',
    'books_technical': 'books/culture/leisure',
    'home_construction': 'DoItYourself',
    'musical_instruments': 'music',
    'furniture_living_room': 'house/decoration/furnitures',
    'construction_tools_lights': 'DoItYourself',
    'industry_commerce_and_business': 'other',
    'food': 'food',
    'art': 'books/culture/leisure',
    'furniture_bedroom': 'house/decoration/furnitures',
    'books_general_interest': 'books/culture/leisure',
    'construction_tools_safety': 'DoItYourself',
    'fashion_underwear_beach': 'fashion/luggage',
    'fashion_sport': 'fashion/luggage',
    'signaling_and_security': 'other',
    'computers': 'electronics',
    'christmas_supplies': 'house/decoration/furnitures',
    'fashio_female_clothing': 'fashion/luggage',
    'home_appliances_2': 'electronics',
    'books_imported': 'books/culture/leisure',
    'drinks': 'food',
    'cine_photo': 'books/culture/leisure',
    'la_cuisine': 'books/culture/leisure',
    'music': 'music',
    'home_comfort_2': 'house/decoration/furnitures',
    'small_appliances_home_oven_and_coffee': 'electronics',
    'cds_dvds_musicals': 'multimedia',
    'dvds_blu_ray': 'multimedia',
    'flowers': 'house/decoration/furnitures',
    'arts_and_craftmanship': 'books/culture/leisure',
    'diapers_and_hygiene': 'hygiene/beauty',
    'fashion_childrens_clothes': 'fashion/luggage',
    'security_and_services': 'other'
}

In [ ]:
category["Big_Categ"] = detect_words(category["product_category_name_english"],
                                     BIG_CAT)

In [ ]:
category["Big_Categ"].unique()

In [ ]:
category.head()

Observations:

- aucune valeur manquante
- une catégorie plus large avec moins de modalités a été crééé

## Consolidation des données (par order_id) <a class="anchor" id="conso"></a>

### Merge

Dans cette partie nous allons relier les différents datasets entre-eux en se basant sur le schéma des données. Nous nous assurons à chaque jointure qu'il y a bien une ligne par commande.

In [ ]:
# Merge orders avec customers
df_conso = pd.merge(left=orders, right=customers[['customer_id',
                                                  'customer_unique_id',
                                                  'customer_zip_code_prefix']],
                    how='left', on='customer_id')
shape_total_nan(df_conso)
df_conso.head()

In [ ]:
# Merge avec geo
df_conso = pd.merge(left=df_conso,
                    right=geo[['geolocation_zip_code_prefix',
                               'zone',
                               'geolocation_lat',
                               'geolocation_lng']],
                    how='left',
                    left_on='customer_zip_code_prefix',
                    right_on='geolocation_zip_code_prefix')

df_conso.drop(columns=['customer_zip_code_prefix',
                       'geolocation_zip_code_prefix'], inplace=True)

df_conso.rename(columns={'geolocation_lat': 'customer_lat',
                         'geolocation_lng': 'customer_lon',
                         'zone': 'customer_zone'}, inplace=True)

shape_total_nan(df_conso)

df_conso.head()

In [ ]:
# Merge orders avec reviews
df_conso = pd.merge(left=df_conso, right=reviews, how='left', on='order_id')
shape_total_nan(df_conso)
df_conso.head()

In [ ]:
# Merge avec payments
df_conso = pd.merge(left=df_conso, right=payments_merge,
                    how='left', on='order_id')
shape_total_nan(df_conso)
df_conso.head()

In [ ]:
# Merge avec order_items
df_conso = pd.merge(left=df_conso, right=items_merge,
                    how='left', on='order_id')
shape_total_nan(df_conso)
df_conso.head()

In [ ]:
# Merge avec products
products_merge.rename(columns={'product_id': 'art_pref'},
                      inplace=True)
df_conso = pd.merge(left=df_conso, right=products_merge,
                    how='left', on='art_pref')
df_conso.rename(columns={'product_category_name': 'cat_pref'},
                inplace=True)
shape_total_nan(df_conso)
df_conso.head()

In [ ]:
# Merge avec category_name
category.rename(columns={'product_category_name': 'cat_pref',
                         'product_category_name_english': 'cat_pref_en',
                         'Big_Categ': 'Big_Categ_pref'},
                inplace=True)
df_conso = pd.merge(left=df_conso, right=category, how='left',
                    on='cat_pref')
shape_total_nan(df_conso)
df_conso.head()

In [ ]:
# Merge avec sellers
sellers.rename(columns={'seller_id': 'vendeur_pref'},
               inplace=True)
df_conso = pd.merge(left=df_conso, right=sellers[['vendeur_pref',
                                                  'seller_zip_code_prefix']],
                    how='left', on='vendeur_pref')
shape_total_nan(df_conso)
df_conso.head()

In [ ]:
# Merge avec geo
df_conso = pd.merge(left=df_conso,
                    right=geo[['geolocation_zip_code_prefix',
                               'zone',
                               'geolocation_lat',
                               'geolocation_lng']],
                    how='left',
                    left_on='seller_zip_code_prefix',
                    right_on='geolocation_zip_code_prefix')

df_conso.drop(columns=['seller_zip_code_prefix',
                       'geolocation_zip_code_prefix'], inplace=True)

df_conso.rename(columns={'geolocation_lat': 'seller_lat',
                         'geolocation_lng': 'seller_lon',
                         'zone': 'seller_zone'}, inplace=True)

shape_total_nan(df_conso)

df_conso.head()

In [ ]:
# Vérification que la table contient bien une ligne par commande
df_conso.shape[0] == df_conso['order_id'].nunique()

### Création de nouveaux indicateurs

Nous allons créer dans ce dataset consolidé d'autres indicateurs qui nous paraissent importants:
- durée de la livraison en jours entre la date où le client a passé commande et la date réelle de livraison 
- différence en jours entre la date estimée de livraison et la date réelle de livraison
- temps en jours que met le client avant de rédiger un commentaire à partir du moment où il reçoit le lien
- si la commande a été livrée (on supprimera les commandes non encore livrées)
- distance entre le vendeur et le client

In [ ]:
# Transformation des dates au format datetime
df_conso['order_purchase_timestamp'] = pd.to_datetime(
    df_conso['order_purchase_timestamp'])
df_conso['order_delivered_customer_date'] = pd.to_datetime(
    df_conso['order_delivered_customer_date'])
df_conso['order_estimated_delivery_date'] = pd.to_datetime(
    df_conso['order_estimated_delivery_date'])
df_conso['review_creation_date'] = pd.to_datetime(
    df_conso['review_creation_date'])
df_conso['review_answer_timestamp'] = pd.to_datetime(
    df_conso['review_answer_timestamp'])

In [ ]:
# Création des indicateurs
df_conso['dur_livr_jrs'] = (df_conso[
    'order_delivered_customer_date'] - df_conso[
    'order_purchase_timestamp']).dt.days

df_conso['diff_estime_livr_jrs'] = (df_conso[
    'order_estimated_delivery_date'] - df_conso[
    'order_delivered_customer_date']).dt.days

df_conso['redac_com_jrs'] = (df_conso[
    'review_answer_timestamp'] - df_conso[
    'review_creation_date']).dt.days

In [ ]:
# Commande livrée ou non
df_conso.loc[df_conso['dur_livr_jrs'].isna(), 'com_livree'] = 0
df_conso.loc[~df_conso['dur_livr_jrs'].isna(), 'com_livree'] = 1

In [ ]:
# Suppression des commandes non livrées
com_non_livrees = df_conso[df_conso['com_livree'] == 0]['order_id'].nunique()
print(f"Commandes non livrées: {com_non_livrees}")
df_conso = df_conso[df_conso['com_livree'] == 1]

In [ ]:
# Fonction qui calcule la distance
def haversine_distance(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    newlon = lon2 - lon1
    newlat = lat2 - lat1

    haversine_formula = np.sin(
        newlat/2.0)**2 + np.cos(
        lat1) * np.cos(
        lat2) * np.sin(
        newlon/2.0)**2

    dist = 2 * np.arcsin(np.sqrt(haversine_formula))
    km = 6367 * dist
    return km

In [ ]:
# Calcul de la distance entre le client et le vendeur
df_conso['distance_km'] = haversine_distance(df_conso['seller_lon'],
                                             df_conso['seller_lat'],
                                             df_conso['customer_lon'],
                                             df_conso['customer_lat'])

In [ ]:
# Suppression des colonnes non utilisées
df_conso.drop(columns=['order_status',
                       'order_approved_at',
                       'order_delivered_carrier_date',
                       'order_delivered_customer_date',
                       'order_estimated_delivery_date',
                       'customer_lat',
                       'customer_lon',
                       'review_id',
                       'review_comment_title',
                       'review_comment_message',
                       'review_creation_date',
                       'review_answer_timestamp',
                       'art_pref',
                       'vendeur_pref',
                       'cat_pref',
                       'seller_lat',
                       'seller_lon',
                       'com_livree'], inplace=True)

### Données manquantes

Le nombre de lignes avec données manquantes étant relativement faible, nous décidons de les supprimer au lieu de rajouter du bruit.

In [ ]:
# Nombre et % de NaN

def NaN_nb_percent(df):
    '''Fonction qui retourne le taux de remplissage et
    le nombre de valeurs manquantes de chaque variable du dataset.'''

    var_dict = {}

    for c in df.columns:
        var_dict[c] = []
        var_dict[c].append(round((df[c].notna().sum()/df.shape[0])*100, 2))
        var_dict[c].append(df[c].isna().sum())

    return pd.DataFrame.from_dict(
        data=var_dict,
        orient='index',
        columns=['Tx de remplissage', 'Nb NaN']).sort_values(
        by='Nb NaN', ascending=False)

In [ ]:
# Taux de remplissage des colonnes
nan = NaN_nb_percent(df_conso)
nan[nan['Nb NaN'] > 0]

In [ ]:
# Suppression des lignes avec au moins un NaN
df_conso.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)

In [ ]:
df_conso.shape

In [ ]:
# Suppression de variables
del geo_dup
del orders
del reviews
del items
del payments
del customers
del products
del items_dup
del geo
del customers_orders
del zip_city
del zip_mean
del payments_dup
del sellers
del reviews_dup
del order_id_dup

### Analyse univariée par commande

In [ ]:
describe_variables(df_conso)

In [ ]:
# Période analysée
print(df_conso['order_purchase_timestamp'].min())
print(df_conso['order_purchase_timestamp'].max())

La période s'étend du 3/10/2016 au 29/08/2018.

In [ ]:
def univ_plots_hist(dataframe, liste_col_quanti, nb_lignes, nb_col, nb_bins):

    for i, c in enumerate(liste_col_quanti, 1):
        ax = fig.add_subplot(nb_lignes, nb_col, i)
        ax.hist(dataframe[c], bins=nb_bins, color='#b8b8d2')
        ax.set_title(c, fontsize=10)
        ax.title.set_fontweight('bold')
        plt.xticks(rotation=45, ha='right')

    plt.tight_layout(w_pad=2, h_pad=2)
    plt.show

In [ ]:
col_quanti = ['review_score', 'facil_paiement_max', 'nb_art', 'nb_art_diff',
              'nb_vendeurs', 'prix_tot_art',
              'prix_transport', 'prix_art_min', 'prix_art_max',
              'dur_livr_jrs', 'diff_estime_livr_jrs', 'redac_com_jrs',
              'distance_km']

fig = plt.figure(figsize=(15, 10))
univ_plots_hist(df_conso, col_quanti, 4, 4, 100)

In [ ]:
def univariate_plots_box(dataframe, liste_col_quanti, nb_lignes, nb_col):
    for i, c in enumerate(liste_col_quanti, 1):
        ax = fig.add_subplot(nb_lignes, nb_col, i)
        ax = sns.boxplot(data=dataframe, x=c, showfliers=True,
                         color='#b8b8d2')
        ax.set_title(c)
        ax.title.set_fontweight('bold')
        plt.xticks(rotation=45, ha='right')

    plt.tight_layout(w_pad=2, h_pad=2)
    plt.show

In [ ]:
fig = plt.figure(figsize=(15, 10))
univariate_plots_box(df_conso, col_quanti, 4, 4)

In [ ]:
def univariate_plots_quali(dataframe, liste_col_quali, nb_lignes, nb_col):
    for i, c in enumerate(liste_col_quali, 1):
        ax = fig.add_subplot(nb_lignes, nb_col, i)
        modalites = dataframe[c].value_counts()
        n_modalites = modalites.shape[0]

        if n_modalites > 15:
            modalites[0:15].plot.bar(color='#b8b8d2', edgecolor='black', ax=ax)

        else:
            modalites.plot.bar(color='#b8b8d2', edgecolor='black')

        ax.set_title(f'{c} ({n_modalites} modalités)',
                     fontweight='bold',
                     fontsize=10)
        labels = [item.get_text() for item in ax.get_xticklabels()]
        short_labels = [lab[0:7] + '.' if len(lab) > 7
                        else lab for lab in labels]
        ax.axes.set_xticklabels(short_labels)
        plt.xticks(rotation=45, ha='right')

    plt.tight_layout(w_pad=2, h_pad=2)
    plt.show

In [ ]:
col_quali = ['customer_zone', 'seller_zone', 'moy_paiement_pref',
             'Big_Categ_pref', 'cat_pref_en']

fig = plt.figure(figsize=(15, 6))
univariate_plots_quali(df_conso, col_quali, 2, 3)

- Les clients notent plutôt favorablement lorsqu'ils passent une commande avec une majorité de 5/5
- Le montant des commandes est généralement peu élevé avec un montant moyen par commande de 137 Réals + 27 Réals de livraison
- Le nombre de facilités de paiement tourne autour de 3 en moyenne mais peuvent être beaucoup plus important
- La majorité des clients sélectionne un article par commande
- Il y a en général un vendeur unique par commande
- Il faut 12 jours en moyenne au client pour être livré
- Il y a en général 11 jours de retard dans les livraisons mais parfois le livreur est en avance
- Le client met 2 jours et demi en moyenne par commande pour rédiger un commentaire
- Les vendeurs se situent en moyenne à 600 km du client
- La majorité des commandes sont passées par des clients basés dans le Sud Est du Brésil
- La majorité des vendeurs se situe également dans le Sud Est
- Les clients paient majoritairement par carte de crédit
- Les produits les plus commandés sont les produits de maison/décoration/fournitures puis l'électronique

### Analyse bivariée / multivariée par commande

#### Variables quantitatives

In [ ]:
# Corrélation
corr = df_conso[col_quanti].corr()

# Création d'un masque
mask = np.triu(corr)

# Taille du graph
plt.subplots(figsize=(12, 5))

# Colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Heatmap avec masque
sns.heatmap(corr, annot=True, mask=mask, cmap=cmap)
plt.xticks(rotation=30, ha='right')
plt.title("Matrice de corrélation entre les variables quantitatives\n",
          fontsize=16)

- La note attribuée à la commande est corrélée aux nombre de jours de retard de livraison et au nombre de jours de livraison
- Le montant total de la commande est corrélé au nombre de facilités de paiement
- Le montant du transport est lié au prix de l'article et à la distance entre le client et le vendeur

#### Une variable quantitative et une qualitative

In [ ]:
infl_note = ['prix_tot_art', 'prix_transport',
             'dur_livr_jrs', 'diff_estime_livr_jrs']

fig = plt.figure(figsize=(15, 9))

%pycodestyle_off
for i, c in enumerate(infl_note, 1):
    ax = fig.add_subplot(2, 2, i)
    meanprops = {'marker':'o', 'markeredgecolor':'black','markerfacecolor':'firebrick'}
    ax = sns.boxplot(data = df_conso, y = c, x = 'review_score',
                     showfliers=False,
                     showmeans=True,
                     meanprops=meanprops)
    plt.suptitle('Dispersion des variables quantitatives en fonction de la note', fontsize=16,
             fontweight='bold')

    ax.title.set_fontweight('bold')
    
plt.tight_layout(w_pad=2, h_pad=2)
%pycodestyle_on

La note associée à la commande semble être surtout liée aux variables en rapport avec la livraison de la commande.
Les clients attribuent les meilleures notes lorsque le prix du transport est plus faible, les délais de livraison plus courts, et la date de livraison respectée ou en avance.

#### Variables qualitatives

In [ ]:
def chi2_test(data, var_quali_1, var_quali_2,
              palette=sns.color_palette('pastel')):
    palette = palette
    # Table de contingence
    tab_cont = pd.crosstab(data[var_quali_1], data[var_quali_2])
    plt.figure(figsize=(12, 10))
    plt.title(f'Table de contingence {var_quali_1} / {var_quali_2}',
              fontsize=12, fontweight='bold')
    sns.heatmap(tab_cont, cmap="YlGnBu", annot=True, cbar=False, fmt="d")
    plt.xticks(rotation=45, ha='right')
    plt.show()
    # Diagramme en barres selon les profils colonnes
    tab_cont_prop = pd.crosstab(data[var_quali_1], data[var_quali_2],
                                normalize="index")
    modalites = len(tab_cont.columns)
    if modalites <= 5:
        with sns.color_palette(palette):
            tab_cont_prop.plot(kind='bar', stacked=True,
                               figsize=(15, 9), width=0.8)
            plt.legend(loc="upper left", ncol=6)
            plt.xlabel(var_quali_1)
            plt.ylabel("Proportion")
            plt.xticks(rotation=45, ha='right')
            plt.title('Diagramme en barres selon les profils colonnes',
                      fontsize=12, fontweight='bold')
            plt.show()

        if len(np.where(tab_cont <= 5)[0]) == 0:
            print('Chaque effectif >= 5 => Test du Chi2 applicable')
            # Running Chi2 test
            print('----------------------------------------------------------')
            st_chi2, st_p, st_dof, st_exp = chi2_contingency(tab_cont)
            print(f"Chi2: {st_chi2}")
            print(f"Degrees of freedom: {st_dof}")
            print('----------------------------------------------------------')
            if st_p < 0.05:
                print(f"pvalue: {st_p} < 0.05 => var dépendantes")
            else:
                print(f"pvalue: {st_p} > 0.05 => var indépendantes")
        else:
            print("Au moins un effectif de la table de contingence < 5")
            print("Test du Chi2 d'indépendance")
            # Running Chi2 Independance
            print('----------------------------------------------------------')
            expected, observed, stat = pg.chi2_independence(data,
                                                            var_quali_1,
                                                            var_quali_2)
            print(stat)
            print('----------------------------------------------------------')
            if stat['pval'][0] < 0.05:
                print(f"pvalue: {stat['pval'][0]} < 0.05 = >var dépendantes")
            else:
                print(f"pvalue: {stat['pval'][0]} > 0.05 => var indépendantes")
    else:
        if len(np.where(tab_cont <= 5)[0]) == 0:
            print('Chaque effectif de la table de contingence >= 5')
            print('Test du Chi2 applicable')
            # Running Chi2 test
            print('----------------------------------------------------------')
            st_chi2, st_p, st_dof, st_exp = chi2_contingency(tab_cont)
            print(f"Chi2: {st_chi2}")
            print(f"Degrees of freedom: {st_dof}")
            print('----------------------------------------------------------')
            if st_p < 0.05:
                print(f"pvalue: {st_p} < 0.05 => var dépendantes")
            else:
                print(f"pvalue: {st_p} > 0.05 => var indépendantes")
        else:
            print("Au moins un effectif de la table de contingence < 5")
            print("Test du Chi2 d'indépendance")
            # Running Chi2 Independance
            print('----------------------------------------------------------')
            expected, observed, stat = pg.chi2_independence(data,
                                                            var_quali_1,
                                                            var_quali_2)
            print(stat)
            print('----------------------------------------------------------')
            if stat['pval'][0] < 0.05:
                print(f"pvalue: {stat['pval'][0]} < 0.05 => var dépendantes")
            else:
                print(f"pvalue: {stat['pval'][0]} > 0.05 => var indépendantes")

In [ ]:
chi2_test(df_conso, 'Big_Categ_pref', 'review_score')

Le test du Chi2 confirme l'hypothèse d'association entre les variables Big_Categ_pref et review_score.

Les meilleures notes semblent être attribuées aux aliments, musique, animaux et jouets et les moins bonnes aux catégories multimédia, maison/décoration, électronique etc.

In [ ]:
chi2_test(df_conso, 'moy_paiement_pref', 'review_score')

Le test du Chi2 confirme l'hypothèse d'association entre les variables moy_paiement_pref et review_score. Cependant, la visualisation du diagramme en barres selon les profils colonnes ne nous laisse pas vraiment penser que la satisfaction dépende du moyen de paiement préféré.

In [ ]:
chi2_test(df_conso, 'customer_zone', 'review_score')

Le test du Chi2 confirme l'hypothèse d'association entre les variables customer_zone et review_score. Les clients du centre est, nord et nord est semblent plus mécontents que les clients du sud et nord ouest par exemple.

## Consolidation des données (par customer_unique_id) <a class="anchor" id="tab_cli"></a>

Le customer_unique_id représente l'identifiant client pour toutes les commandes. Nous allons passer par la table intermédiaire sur les commandes pour pouvoir calculer les indicateurs aggrégés suivants par **client** (dont les indicateurs RFM utilisés fréquemment dans les segmentations marketing: **Recency** (jours passés depuis le dernier achat), **Frequency** (nombre total de commandes) et **Monetary** (montant total dépensé)):
- catégorie de produits la plus achetée
- date de la dernière commande (nous calculerons ensuite la différence en jours avec la date du jour pour obtenir la **Recency**)
- mode de paiement préféré
- nombre total de commandes (**Frequency**)
- note moyenne des commandes
- nombre moyen de facilités de paiement
- nombre d'articles moyens par commande
- nombre d'articles achetés
- nombre d'articles différents achetés
- montant total dépensé (**Monetary**)
- montant moyen par commande
- prix moyen du transport
- prix moyen des articles par commande
- prix de l'article le plus cher acheté
- prix de l'article le moins cher acheté
- durée moyenne de livraison en jours
- respect de la date de livraison moyenne en jour (négatif = retard de livraison et positif = avance de livraison)
- nombre de jours moyens pour rédiger un commentaire
- distance moyenne entre le client et le vendeur

In [ ]:
df_customers = df_conso.groupby(['customer_unique_id'],
                                as_index=False, dropna=False).agg(
    cat_pref=('cat_pref_en', lambda x: x.mode()[0]),
    big_cat_pref=('Big_Categ_pref', lambda x: x.mode()[0]),
    date_dern_com=('order_purchase_timestamp', 'max'),
    moy_paiement_pref=('moy_paiement_pref', lambda x: x.mode()[0]),
    Frequency=('order_id', 'nunique'),
    note_moy_com=('review_score', 'mean'),
    facil_paiement_max_moy_com=('facil_paiement_max', 'mean'),
    nb_art_moy_com=('nb_art', 'mean'),
    nb_art_tot_ach=('nb_art', 'sum'),
    nb_art_diff_ach=('nb_art_diff', 'sum'),
    Monetary=('prix_tot_art', 'sum'),
    montant_moy_com=('prix_tot_art', 'mean'),
    prix_trans_moy_com=('prix_transport', 'mean'),
    prix_moy_art_com=('prix_tot_art', 'mean'),
    prix_max_art=('prix_art_max', 'max'),
    prix_min_art=('prix_art_min', 'min'),
    jrs_livr_moy_com=('dur_livr_jrs', 'mean'),
    respect_date_livr_moy_com=('diff_estime_livr_jrs', 'mean'),
    jrs_redac_review_moy_com=('redac_com_jrs', 'mean'),
    dist_cli_vend_moy_km=('distance_km', 'mean'))

In [ ]:
# Création table avec la Zone la plus récente du client
tab_zone_cli = df_conso.groupby(['customer_unique_id', 'customer_zone'],
                                as_index=False, dropna=False).agg(
    order_purchase_timestamp=('order_purchase_timestamp', 'max')).sort_values(
    by='order_purchase_timestamp', ascending=False)

# Nombre de lignes ayant le même customer_unique_id
print(f"Nb lignes: {tab_zone_cli.duplicated('customer_unique_id').sum()}")

# On converve la zone la plus récente
tab_zone_cli = tab_zone_cli.drop_duplicates('customer_unique_id',
                                            keep='first')

In [ ]:
# Ajout de la zone la plus récente du client
df_customers = pd.merge(left=df_customers, right=tab_zone_cli,
                        how='left', on='customer_unique_id')

In [ ]:
# Vérification que la table contient bien une ligne par client
df_customers.shape[0] == df_conso['customer_unique_id'].nunique()

In [ ]:
# Date de commande la plus récente:
df_customers['date_dern_com'].max()

Nous allons calculer la "Recency" en partant du principe que nous sommes le 31/08/2018, 23h59.

In [ ]:
# Calcul de la "Recency"
now = dt.datetime(year=2018, month=8, day=31, hour=23, minute=59, second=0)
now

In [ ]:
df_customers['Recency'] = (now - df_customers[
    'date_dern_com']).dt.days

In [ ]:
df_customers.head()

In [ ]:
# Taux de remplissage des colonnes
NaN_nb_percent(df_customers)

### Analyse des indicateurs RFM

In [ ]:
describe_variables(df_customers)

In [ ]:
col_quanti = ['Recency', 'Frequency', 'Monetary']

fig = plt.figure(figsize=(15, 5))
univ_plots_hist(df_customers, col_quanti, 1, 3, 100)

In [ ]:
fig = plt.figure(figsize=(15, 5))
univariate_plots_box(df_customers, col_quanti, 1, 3)

- Le nombre de jours passés depuis le dernier achat se situe entre 2 et près de 700 jours avec une moyenne de 239 jours et une médiane de 219 jours
- Les clients effectuent 1 commande en moyenne avec un maximum de 14 commandes
- Les clients ont dépensé en moyenne 141 Réals

In [ ]:
# Liste des clients ayant passé plusieurs commandes
cli_multi_order = df_customers[df_customers['Frequency'] > 1]
cli_multi_order = cli_multi_order['customer_unique_id'].unique().tolist()

# % de clients ayant passé plusieurs commandes
nb_cli_multi = len(cli_multi_order)
nb_cli = df_customers['customer_unique_id'].nunique()
nb_cli_multi_percent = nb_cli_multi / nb_cli * 100
print(f"% clients avec plusieurs commandes {round(nb_cli_multi_percent,2)}")

In [ ]:
nb_cli_multi_percent

On retombe bien sur les 3% de clients indiqués dans l'énoncé.

In [ ]:
# Export des données
df_customers.to_csv('data/df_customers.csv')
df_conso.to_csv('data/df_conso.csv')